W tym zadaniu przećwiczymy PCA oraz poznamy inne metody redukcji wymiarowości.

W poprzednich dwóch zadaniach ekstrahowaliśmy cechy z sygnału podzielonego na ramki, uzyskując obraz (macierz 2D) opisujący sygnał. Jest to bardzo często stosowane w analizie sygnałów podejście, ale nie jedyne.

Z sygnałów można też ekstrahować inne parametry, np. parametry czasowe sygnału (częstotliwość przejść przez zero, środek ciężkości sygnału), parametry widmowe (momenty widmowe, kurtoza i skośność widma), parametry formantowe (częstotliwości formantów, stosunek ich amplitud) i wiele innych. Zbiór różnych parametrów sygnałów, które dają bardzo dobre rezultaty w klasyfikacji sygnałów akustycznych jest zawarty w bibliotece [openSMILE](https://audeering.github.io/opensmile/index.html).

OpenSMILE pozwala wyznaczyć predefiniowane zestawy parametrów opisujących dźwięk lub wideo - skorzystamy z zestawu 6373 parametrów opisujących pojedynczy sygnał (jest to zestaw cech i ich pochodnych używany w Interspeech Computational Paralinguistics Challenge; dla zainteresowanych opis: https://doi.org/10.3389/fpsyg.2013.00292 - tabele 2 i 3).

In [ ]:
# jesli pracujemy w Colabie, konieczna jest instalacja biblioteki
!pip install opensmile
import opensmile
from sklearn.preprocessing import StandardScaler
import numpy as np

In [1]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

feats = smile.process_file('dane_testowe/1-phrase.wav')
feats.shape

(1, 6373)

Wykorzystując napisany wcześniej kod, wylicz parametry wszystkich sygnałów zawartych w pliku `phrase_files.csv`. Podziel dane na zbiór treningowy i testowy.

Ze względu na zróżnicowanie parametrów, musimy przeprowadzić standaryzację:

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Na ustandaryzowanych danych wytrenuj używany już wcześniej klasyfikator k najbliższych sąsiadów i wylicz metryki.

Warto ten krok zdefiniować jako funkcję - będziemy korzystać z niego kilka razy w tym notatniku.

6373 cechy opisujące jeden sygnał to dość dużo. Żeby zmniejszyć liczbę tych cech należy zastosować jedną z metod redukcji wymiarowości. Wpłynie to na zmniejszenie skomplikowania niektórych rodzajów klasyfikatorów, przyspieszy proces uczenia i pozwoli zmniejszyć przeuczenie modelu.

Zaczniemy od metody bazującej na wartości statystyki F. W bibliotece sklearn jest to funkcja [`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest).

Wybierz liczbę cech, które chcesz zachować. Funkcja wybierze ze wszystkich 6373 cech tylko tyle, ile sprecyzujesz, w taki sposób, by jak najlepiej opisywać zmienność i różnicować obiekty.

Liczbę cech można optymalizować, ale tym zajmiemy się kiedy indziej. Dzisiaj spróbuj ręcznie dobrać ich liczbę tak, by wyniki klasyfikacji były zadowalające.

In [ ]:
from sklearn.feature_selection import SelectKBest

liczba_cech = #wpisz tu wartość
selector = SelectKBest(k=liczba_cech)
X_train_Kbest = selector.fit_transform(X_train)
X_test_Kbest = selector.transform(X_test)

Wytrenuj klasyfikator na zredukowanej liczbie cech. Jakie wartości metryk wyszły teraz? Czy są lepsze czy gorsze niż wcześniej?

Kolejną funkcją wartą poznania jest metoda bardzo podobna do `SelectKBest`, czyli [`SelectPercentile`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile). Działa na tej samej zasadzie, ale zamiast określać konkretną liczbę cech, które chcemy zachować, określamy jaka część cech ma być zachowana, np. 40%.

In [ ]:
from sklearn.feature_selection import SelectPercentile

percentile = #wpisz tu wartość
selector = SelectPercentile(percentile=percentile)
X_train_percent = selector.fit_transform(X_train)
X_test_percent = selector.transform(X_test)


Ostatnią metodą, którą dzisiaj poznamy jest [rekursywna eliminacja cech](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) (RFE, ang. recursive feature elimination). Polega na tym, że estymator początkowo uczony jest na całym zbiorze danych i tworzony jest ranking cech w oparciu o to, jak bardzo są ważne podczas estymacji. Cechy najmniej istotne są usuwane i cały proces jest powtarzany. Usuwanie cech trwa tak długo, aż uzyskamy taką liczbę, jak sprecyzowana.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

n_features = #wpisz tu wartość
step = #wpisz tu wartość
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=n_features, step=step) 
"""step - jeżeli >=1, to jest to liczba cech do usunięcia w danej iteracji,
jeżeli 0<step<1 - część cech do usunięcia.
Analogicznie działa parametr n_features_to_select"""



Powtórz proces uczenia i predykcji na cechach uzyskanych metodą RFE. Jakie wartości metryk wyszły tym razem? 

Na koniec wykonaj PCA na cechach z openSMILE, wykorzystując kod z poprzedniego zadania. Porównaj wyniki z pozostałymi metodami.